# python C 확장

- 파이썬에서 외부 라이브러리를 쓰기 위해서는 확장 모듈을 만들고 그 속에서 외부 라이브러리를 호출
- CPython이 개발자에게 제공하는 흥미로운 기능은 C 코드와 Python의 인터페이스 용이성
- 개발자가 Python 코드에서 C 함수를 호출하는 데 사용하는 세 가지 주요 방법
    - ctypes, SWIG 및 Python / C API
    - 각 방법마다 장점과 단점이 있음
- 참고로.... [C와 C++의 차이](https://codingcoding.tistory.com/287)
    
### 왜 C와 파이썬을 연결하는 것이 필요한가?
- 속도 향상이 필요할 때: C가 파이썬보다 50배 빠르다
- 특정 레거시 C라이브러리가 존재할 때: 이미 당신이 원하는 대로 작동하므로 파이썬으로 다시 작성하고 싶지 않음
- 특정 저수준 자원 액세스 - 메모리에서 파일 인터페이스
- 그냥 하고 싶어서..?

### + 확장 모듈이 필요한 이유
1. 새로운 내장 객체 타입을 구현
    - 확장 모듈을 이용하여 파이썬에서 C/C++ 라이브러리 함수 혹은 시스템 콜을 할 수 있도록 함

2. 파이썬의 연산 처리 속도
    - 파이썬의 연산 처리 속도가 C에 비해서 약간 느리게 동작
    - 사용자 인터페이스, 문자열 처리 및 자료형은 파이썬을 사용, 빠른 연산을 요구하는 작업은 C/C++의 모듈에서 실행하는 프로그램을 구현할 수 있음

3. 코드 은닉
    - 파이썬 코드를 C/C++모듈이나 C확장형으로 구현 시, 작성한 파이썬 코드(*.py)들의 핵심적인 부분을 공개하지 않고 배포가능
    - 파이썬은 인터프리터 언어 처럼 동작, 코드가 직접적으로 나타남

# 1. ctypes

- Python ctypes 모듈은 Python에서 C 함수를 호출하는 가장 쉬운 방법
- 파이선에서 제공하는 파이썬 ctypes 모듈을 이용하면 C의 데이터 타입이나, DLL 혹은 공유 라이브러리(shared library)의 함수를 직접 사용 가능
- 수정할 필요없이 C 공유 라이브러리를 호출 가능
- C 부분은 만질 필요도 없으므로 가장 단순한 방법

- 아래 예제와 같이 파이썬 ctypes를 임포트하는 것으로 모든 준비가 끝남

In [ ]:
import ctypes

# 아직 실행되진 않음
print(ctypes.cdll.msvcrt)

- 예제를 위해 우선 c코드를 컴파일해야 함

``` C
// 2개의 수를 더하는 C 샘플 코드 - int and floats
// add.c로 저장

#include <stdio.h>

int add_int(int, int);
float add_float(float, float);

int add_int(int num1, int num2){
    return num1 + num2;
}

float add_float(float num1, float num2){
    return num1 + num2;
}
```

- 다음으로 C 파일을 .so 파일 (Windows의 DLL)로 컴파일
    - 그러면 adder.so 파일이 생성

``` bash
#리눅스용
$  gcc -shared -Wl,-soname,adder -o adder.so -fPIC add.c

#맥용
$ gcc -shared -Wl,-install_name,adder.so -o adder.so -fPIC add.c
```

## 예제 설명
- 이 예제의 C파일에서는 두 개의 함수를 가지고 있고, 하나는 두 개의 정수를 더하고 다른 하나는 실수형을 더하는 단순한 형태
- 파이썬 파일에서는 먼저 ctypes 모듈을 가져옴
- 그리고 ctypes모듈의 CDLL 함수가 우리가 생성한 공유 라이브러리 파일을 읽기 위해 사용됨
- **이제 C라이브러리 안에 정의된 함수들을 adder 변수를 통해서 사용할 수 있음(!)**
    - adder.add_int()가 호출 되면 내부적으로 add_int C 함수가 호출
- ctypes 인터페이스를 사용하면 기본적으로 C 함수를 호출하는 동안 네이티브 파이썬 정수 및 문자열을 사용할 수 있음
- boolean 또는 float와 같은 다른 유형의 경우 올바른 데이터 타입을 사용해야 함(**사용할 수 있는 타입들 아래 표 참고**)
    - 이는 매개 변수를 adder.add_float ()에 전달할 때 확인 가능
- 먼저 파이썬 10진수 값에서 필요한 c_float 유형을 만든 다음 C 코드에 대한 인수로 사용
- **ctypes의 장단점**
    - 이 방법은 간단하고 명료하지만 제한적임
    - 예를 들어 C의 객체를 조작할 수 없다는 단점

In [6]:
import ctypes

#load the shared object file
adder = ctypes.CDLL('./adder.so')

#Find sum of integers
res_int = adder.add_int(4,5)
print("Sum of 4 and 5 = {}".format(str(res_int)))

#Find sum of floats
a = c_float(5.5)
b = c_float(4.1)

add_float = adder.add_float
add_float.restype = c_float
print("Sum of 5.5 and 4.1 = {}".format(str(add_float(a, b))))

Sum of 4 and 5 = 9
Sum of 5.5 and 4.1 = 9.600000381469727


## ctypes에서 사용할 수 있는 C의 데이터 타입
- 파이썬 ctypes는 외부 함수 호출 말고도 C의 데이터 타입을 쓸 수가 있음
    - 아래는 기본적인 ctypes 함수가 지원하는 타입

<img src="img/Capture.PNG" width="600">

- 아래와 같이 그냥 native python 스크립트에서 ctypes 사용 가능

In [13]:
import ctypes
i = ctypes.c_int(10)
print("ctypes : {}, value : {}".format(i, i.value))

ctypes : c_int(10), value : 10


- 또한 다음 예제와 같이 파이썬 ctypes는 포인터도 지원
    - `pointer` 함수를 이용해 `c_int(10)`의 포인터 값을 가져옴

In [15]:
k = ctypes.c_int(10)
pk = ctypes.pointer(k)
print("position : {}, contents : {}".format(pointer(k), pk.contents))

position : <__main__.LP_c_int object at 0x7f263816ce18>, contents : c_int(10)


- **구조체와 유니온은 파이선에서 class로 표현**
    - 일단 class 객체를 생성하고, `_fields_` 멤버 변수를 이용해 각 멤버 변수를 설정

In [18]:
import ctypes

class POINT(ctypes.Structure):
    _fields_ = [("x", ctypes.c_int), ("y", ctypes.c_int)]

point = POINT(10, 20)
print(point.x, point.y)

point = POINT(y=5)
print(point.x, point.y)

10 20
0 5


# 2. SWIG(Simplified Wrapper and Interface Generator)

- [공식 페이지](http://swig.org/)
- C, C++로 작성된 코드를 Python, Perl, Ruby, Tcl등 다양한 high-level Programming languages로 연결 시켜주는 tool
- C#, Lisp등의 non-scripting 언어도 지원
- 실제로 c++을 다른 프로그래밍 언어로 wrapping하는 것이 쉬운 일은 아님. target lanaguage가 c++의 operator overriding을 지원하지 않을 수도 있고 new/delete 같은 것들을 지원하지 않을 수도 있음
- 또한 c++은 다양한 OOP의 개념을 구체화한 언어이다 보니 그 복잡성이 크고, STL같은 template같은 개념도 등장하는지라 여간해서 c++을 다른 프로그래밍 언어로 포팅하는 것이 쉽지 않음
- 일단 swig는 이 모든 문제를 해결하기 위하여 target language단에서 proxy class를 만들고, 실제 c++ class는 flatted c-like accessor interface로 변경하여 접근하는 방식을 사용

## 2-1. SWIG의 사용 순서
1. C,C++코드를 작성
2. SWIG 인터페이스 파일을 작성
3. 단계1과 2의 파일을 이용하여 타겟 고수준언어의 모듈을 컴파일
4. 해당 언어에서 호출 하여 사용

## 2-2. SWIG로 C++을 python에서 사용하기

- (linux ubuntu 16.04 기준) 우선 swig를 설치해줌

``` bash
sudo apt-get install swig
```

### 1.  아래 간단한 형태의 header 파일을 생성: `hello_cpp.h`
- "hello world" 를 출력해주는 함수

``` c++
#include <iostream>
using namespace std;

class CHello {
public :
        CHello();
        virtual ~CHello();

        void hello1(char *name);
};
```


### 2.  `hello_cpp.cpp`
``` c++
#include "hello_cpp.h"
CHello::CHello() {
}

CHello::~CHello() {
}

void CHello::hello1(char *name)
{
        cout << "Hello " << name << endl;
}
```

### 3. python 모듈 생성: `hello_cpp.i`
- 위의 C 함수를 위한 interface 파일을 만들어야 함
- 이 파일은 swig가 C 함수를 다른 언어와 연결시키기 위한 정보 파일을 만들기 위해서 이용

``` c++
%module hello_cpp
%{
#include "hello_cpp.h"
%}
%include "hello_cpp.h"
```

### 4. 라이브러리 생성
- object 파일을 ld 명령어를 이용해서 공유라이브러리 형태로 만들어 줌
- 아래 command까지 돌려주면 이제 python 에서 모듈로 사용 가능

``` bash
swig -python -c++ hello_cpp.i
g++ -c hello_cpp.cpp hello_cpp_wrap.cxx -I/home/lynn/.virtualenvs/data/include/python3.5m -fPIC
g++ -shared hello_cpp.o hello_cpp_wrap.o -o _hello_cpp.so -fPIC
```

In [2]:
# command창에서 바로 실행하면 나오는데 notebook 환경에서는 프린트가 되지 않음
import hello_cpp
he = hello_cpp.CHello()
he.hello1("world")

- c++ 외에 c와의 연동 예제는 [여기](http://blog.naver.com/PostView.nhn?blogId=wisereign&logNo=30035124764&categoryNo=17&parentCategoryNo=17&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postList&userTopListOpen=true&userTopListCount=5&userTopListManageOpen=false&userTopListCurrentPage=1) 참고

### distutils
- command 창에서 compile하거나 아래처럼 distutils 사용하여 so 파일 생성 가능

In [ ]:
# 실행되진 않는 예제입니다!
from distutils.core import setup, Extension
 
module_hello = Extension('hello',
                    include_dirs = ['usr/include'],
                    libraries = ['boost_python'],
                    library_dirs = ['usr/lib'],
                    sources = ['greet.cpp'])
 
setup (name = 'hello',
       version = '0.1',
       description = 'This is a test of Boost.Python',
       ext_modules = [module_hello])

# 3. python / C API

- 자세한 설명은 생략... ([API 공식 문서](https://translate.googleusercontent.com/translate_c?depth=1&hl=ko&prev=search&rurl=translate.google.com&sl=en&sp=nmt4&u=https://docs.python.org/3/c-api/&xid=17259,15700023,15700186,15700190,15700256,15700259&usg=ALkJrhhPKH6y2zrrAvNOEB_6ANp0nsiWDA))
- 생략하는 이유는 래핑하는 것이 복잡
- python 쪽에 제공하고자 하는 코드가 아래와 같다면,

```c
#include <stdexcept>

char const* greet(unsigned x)
{
    static char const* const msgs[] = { ‘‘hello’’, ‘‘Boost.Python’’, ‘‘world!’’ };
    if (x > 2)
        throw std::range_error(‘‘greet: index out of range’’);
    return msgs[x];
}
```

- 이를 파이썬이 제공하는 'C' interoperability API를 써서 감싸려면(wrapping), 아래와 같이 해야 함

``` c
extern ‘‘C’’ // all Python interactions use ’C’ linkage and calling convention
{
    // Wrapper to handle argument/result conversion and checking
    PyObject* greet wrap(PyObject* args, PyObject * keywords)
    {
        int x;
        if (PyArg_ParseTuple(args, ''i'', &x)) // extract/check arguments
        {
            char const* result = greet(x); // invoke wrapped function
            return PyString_FromString(result); // convert result to Python
        }

        return 0;  // error occurred

    }

    // Table of wrapped functions to be exposed by the module
    static PyMethodDef methods[] = {
            { ‘‘greet’’, greet wrap, METH_VARARGS, ‘‘return one of 3 parts of a greeting’’ }
            , { NULL, NULL, 0, NULL } // sentinel
        };



    // module initialization function
    DL_EXPORT init_hello()

    {
        (void) Py_InitModule(‘‘hello’’, methods); // add the methods to the module
    }
}
```

- 자유도가 높고 c/c++에 익숙한 사람들은 API를 이용하시면 되겠지만 대부분은 그냥 1~2번을 사용

# 마치며
- "파이썬의 동적 타입 시스템은 프로그램에 유연성을 주는 근간이 되며, 반면 C++의 정적 타입 시스템은 효율성의 주춧돌이 되고 있다"
- 상호보완적으로 사용하면 큰 시너지

# Reference
- [9. C/C++와 연동](https://eunguru.tistory.com/84)
- [20. 파이썬 C 확장(작성중)](https://ddanggle.gitbooks.io/interpy-kr/content/c.html)
- [파이썬 CTypes 구조체 자세히 알아보기 (PyThon, 파이선)](https://codingcoding.tistory.com/610)
- [\[Python\] ctypes 를 이용한 shard library 직접 호출](http://egloos.zum.com/mcchae/v/11024689)
- [\[SWIG\] 기본 사용법(리눅스에서 Python에서 사용 예) ](http://blog.daum.net/_blog/BlogTypeView.do?blogid=0G6E3&articleno=7332103&categoryId=263290&regdt=20110612141047)
- [swig로 c++을 python에서 사용하자.](http://blog.naver.com/PostView.nhn?blogId=wisereign&logNo=30036496844&categoryNo=17&parentCategoryNo=17&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postList&userTopListOpen=true&userTopListCount=5&userTopListManageOpen=false&userTopListCurrentPage=1)
- [Boost.Python으로 C++와 Python 같이 쓰기 #01](http://blog.naver.com/PostView.nhn?blogId=lithium81&logNo=80123555915)